In [8]:
import torch

In [9]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your
    [0.55, 0.87, 0.66],  # journey
    [0.57, 0.85, 0.64],  # starts
    [0.22, 0.58, 0.33],  # with
    [0.77, 0.25, 0.10],  # one
    [0.05, 0.80, 0.55]]  # step
)

In [10]:
query = inputs[1] # segundo token é a query
attn_scores = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores[i] = torch.dot(x_i, query) # produto escalar para descobrir similaridade (alinhamento)

print(attn_scores)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [11]:
# Normalização serve para interpretação e estabilidade no treino, passar
# para formato de porcentagens
# Após normalização, temos o tensor attention weights
# Principal objetivo fazer com que a soma dos attention weights seja 1
attn_weights_tmp = attn_scores / attn_scores.sum() # Jeito mais simples é dividir cada um pela soma
print(attn_weights_tmp)

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


In [12]:
# No entanto, é recomendado usar a função softmax para normalização
# Quando temos valores extremos, é ideal que valores pequenos sejam 0
# e valores grandes sejam 1, para não confundir o otimizador durante 
# back propagation, pois ele daria importância para esses pequenos valores

# Sendo xn um attention score qualquer
# Softmax tem uma fórmula e^xn / sum, sendo sum = e^xi + e^xi+1...
# Pytorch tem uma implementação e^(xn - max), sendo max o valor máximo
attn_weights = torch.softmax(attn_scores, dim=0)
print("Attention weights: ", attn_weights)

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
